In [1]:
import cv2
print(cv2.__version__)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import imutils
import sys
import numpy
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pickle
numpy.set_printoptions(threshold=sys.maxsize)

4.1.1


In [2]:
%run commons.py

In [3]:
img_full = cv2.imread('full_image.jpg')
img_bkg = cv2.imread('selected_data/background.jpg')
img_pallet = cv2.imread('selected_data/pallet.jpg')

In [4]:
# Blur
blur_size = 7
img_full = cv2.medianBlur(img_full, blur_size)
img_bkg = cv2.medianBlur(img_bkg, blur_size)
img_pallet = cv2.medianBlur(img_pallet, blur_size)

In [5]:
channels = "HSCMYb"

In [6]:
img_bkg_features = features(img_bkg, channels)
img_pallet_features = features(img_pallet, channels)

X = np.concatenate((img_bkg_features, img_pallet_features))
y = np.concatenate(([0]*len(img_bkg_features), [1]*len(img_pallet_features)))

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)
      
train_index, test_index = sss.split(X, y).__next__()
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

In [7]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
filename = 'blurred_naive_bayes_model.sav'
pickle.dump(gnb, open(filename, 'wb'))

y_pred=gnb.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[598392, 192442],
       [168887, 326170]])

In [8]:
# clf = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=0)
# clf.fit(X_train, y_train)

# y_pred=clf.predict(X_test)
# print(clf.feature_importances_)
# confusion_matrix(y_test, y_pred)